# Natural Computing Project - BiomakerCA - Simulating Plant Reproducibility 

### 1. Overrides

In [1]:
# Overriding the default environment logic with a custom one
import overrides.env_logic_override as env_override
import self_organising_systems.biomakerca.env_logic as env_logic
env_logic.process_energy = env_override.process_energy


# Overriding the default step maker with a custom one
import overrides.step_maker_override as step_maker_override
import self_organising_systems.biomakerca.step_maker as step_maker
step_maker.step_env = step_maker_override.step_env



### 2. Imports

In [2]:
import jax.random as jr
import mediapy as media
import numpy as np
from IPython.display import Video
from jax import vmap
from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import (
    BasicMutator, RandomlyAdaptiveMutator)

from biomaker_utils import perform_evaluation, perform_simulation, start_simulation
from configs.base_config import BaselineConfig



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/jax/_src/api_util.py:231: SyntaxWarning: Jitted function has invalid argnames {'n_steps', 'config', 'n_max_programs', 'mutator'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


### 3. Baseline Configuration

In [3]:
base_config = BaselineConfig()

st_env, env_config = evm.get_env_and_config(base_config.ec_id, width_type=base_config.env_width_type)
env_config.soil_unbalance_limit = base_config.soil_unbalance_limit
agent_logic = BasicAgentLogic(env_config, minimal_net=base_config.agent_model == "minimal")
env_config.specialize_cost = np.asarray([0.035, 0.035]) # added, original: jp.array([0.02, 0.02])

sd = 1e-2 if base_config.mutator_type == "basic" and base_config.agent_model == "basic" else 1e-3
mutator = (
    BasicMutator(sd=sd, change_perc=0.2)
    if base_config.mutator_type == "basic"
    else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)
)

print("\n\nCurrent config:")
print("\n".join("%s: %s" % item for item in vars(env_config).items()))

ku, key = jr.split(base_config.key)
programs = vmap(agent_logic.initialize)(jr.split(ku, base_config.n_max_programs))
programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)

env = st_env

BasicAgentLogic.dsm_num_params = 0
BasicAgentLogic.nsl_num_params = 176
BasicAgentLogic.denm_num_params = 80
BasicAgentLogic.excl_num_params = 41
BasicAgentLogic.repr_num_params = 2
BasicAgentLogic.num_params = 299


Current config:
agent_state_size: 2
etd: DefaultTypeDef: {materials_list: ['VOID', 'AIR', 'EARTH', 'IMMOVABLE', 'SUN', 'OUT_OF_BOUNDS'], types: {'VOID': 0, 'AIR': 1, 'EARTH': 2, 'IMMOVABLE': 3, 'SUN': 4, 'OUT_OF_BOUNDS': 5, 'AGENT_UNSPECIALIZED': 6, 'AGENT_ROOT': 7, 'AGENT_LEAF': 8, 'AGENT_FLOWER': 9}, specialization_idxs: {'AGENT_UNSPECIALIZED': 0, 'AGENT_ROOT': 1, 'AGENT_LEAF': 2, 'AGENT_FLOWER': 3}, agent_types: [6 7 8 9], intangible_mats: [0 1], gravity_mats: [2 6 7 8 9], structural_mats: [6 7 8 9], propagate_structure_mats: [2 3 6 7 8 9], agent_spawnable_mats: [0 1 2], structure_decay_mats: [-1 -1  1  0 -1 -1  5  5  5  5], aging_mats: [6 7 8 9], dissipation_rate_per_spec: [[0.5 0.5],  [1.  1. ],  [1.  1. ],  [1.2 1.2]]}
env_state_size: 6
struct_integrity_cap: 200
abso

In [4]:
### 4. Performing Basic Simulation

In [5]:
# spring_agent_logic = agent_logic

# frame = start_simulation(env, base_config, env_config)
# with media.VideoWriter(
# 	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
# ) as video:
# 	perform_simulation(
# 		env, programs, base_config, env_config, spring_agent_logic, mutator, key, video, frame
# 	)

# Video(base_config.out_file)

In [6]:
# perform_evaluation(env, programs, st_env, env_config, agent_logic, mutator, base_config)

### 5. Simulating Seasons

In [7]:
APPEND_ALL_SEASONS_IN_ONE_VIDEO = False

#### 5.1 Spring

In [8]:
base_config.AIR_DIFFUSION_RATE = 0.09

if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	base_config.out_file = "spring.mp4"

frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	step = perform_simulation(
		env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, season="Autumn"
	)

# if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
# 	Video(base_config.out_file)

Video(base_config.out_file)

Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>


  0%|          | 0/125 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int32 to dtype=uint32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
2024-04-30 14:18:48.678876: E external/xla/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  %reduce.73 = f32[72,128,9,6]{3,2,1,0} reduce(f32[1,72,128,9,6]{4,3,2,1,0} %broadcast.30, f32[] %constant.155), dimensions={0}, to_apply=%region_129.7164, metadata={op_name="jit(step_env)/jit(main)/reduce_sum[axes=(0,)]" source_file="/Users/laurastritzel/Desktop/Radboud/semester 2/Natural Computing/final project/NaCO_project/overrides/step_maker_override.py" source_line=157}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runt

In [13]:
agentTypes = env.state_grid

# Count of each type of agent
zeros = np.count_nonzero(agentTypes == 0)
ones = np.count_nonzero(agentTypes == 1)
twos = np.count_nonzero(agentTypes == 2)
threes = np.count_nonzero(agentTypes == 3)
four = np.count_nonzero(agentTypes == 4)
fives = np.count_nonzero(agentTypes == 5)

print("Count of all 0 in grid: ", zeros)
print("Count of all 1 in grid: ", ones)
print("Count of all 2 in grid: ", twos)
print("Count of all 3 in grid: ", threes)
print("Count of all 4 in grid: ", four)
print("Count of all 5 in grid: ", fives)

Count of all 0 in grid:  46334
Count of all 1 in grid:  0
Count of all 2 in grid:  8958
Count of all 3 in grid:  0
Count of all 4 in grid:  0
Count of all 5 in grid:  0


In [14]:
perform_evaluation(env, programs, st_env, env_config, agent_logic, mutator, base_config)

Extracted 1 programs.


KeyboardInterrupt: 

#### 5.2 Summer

In [7]:
base_config.AIR_DIFFUSION_RATE = 0.1

if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	base_config.out_file = "summer.mp4"
	frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	step = perform_simulation(
		env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Summer"
	)

if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	Video(base_config.out_file)

100%|██████████| 125/125 [00:22<00:00,  5.49it/s]


In [16]:
perform_evaluation(env, programs, st_env, env_config, agent_logic, mutator, base_config)


Extracted 1 programs.
Took 16.3769850730896 seconds
Total number of agents [13268] 13268.0 0.0
Extinction events [0] 0.0 0.0
Number of flowers:  0


#### 5.3 Autumn

In [10]:
base_config.AIR_DIFFUSION_RATE = 0.13

if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	base_config.out_file = "autumn.mp4"
	frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	step = perform_simulation(
		env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Autumn"
	)

if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	Video(base_config.out_file)

100%|██████████| 125/125 [00:32<00:00,  3.90it/s]


#### 5.4 Winter

In [11]:
base_config.AIR_DIFFUSION_RATE = 0.05

if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	base_config.out_file = "winter.mp4"
	frame = start_simulation(env, base_config, env_config)
with media.VideoWriter(
	base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
) as video:
	step = perform_simulation(
		env, programs, base_config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Winter"
	)

Video(base_config.out_file)

 46%|████▋     | 58/125 [00:16<00:18,  3.60it/s]


KeyboardInterrupt: 